In [2]:
import torch
from tape import ProteinBertModel, TAPETokenizer
#unirep
from tape import UniRepModel
#getting the sweet, sweet < 2K AA proteins to use for the TAPE embeddings
import string
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyensembl
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import re
from urllib.request import urlopen
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_protein
from io import TextIOWrapper
import time
import os.path
import matplotlib.pyplot as plt
from os import path
from queue import PriorityQueue
from itertools import combinations
from numpy.random import choice
import random
import sqlite3

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [4]:
def getTransformerMeanSequenceOutput(seq):
    #from github: pooled_output is *not* trained for the transformer
    model = ProteinBertModel.from_pretrained('bert-base')
    tokenizer = TAPETokenizer(vocab='iupac')  # iupac is the vocab for TAPE models, use unirep for the UniRep model
    token_ids = torch.tensor([tokenizer.encode(seq)])
    output = model(token_ids)
    #average embedding for the bert model, average along sequence as recommendd
    #output shape is (1,768)
    return torch.mean(output[0],1)

def getUniRepPooledSequence(sequence):
    #return
    model = UniRepModel.from_pretrained('babbler-1900')
    tokenizer = TAPETokenizer(vocab='unirep')
    token_ids = torch.tensor([tokenizer.encode(sequence)])
    output = model(token_ids)
    return output[1]

In [5]:
seq = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
getTransformerMeanSequenceOutput(seq)



INFO:tape.models.file_utils:https://s3.amazonaws.com/proteindata/pytorch-models/bert-base-config.json not found in cache, downloading to /tmp/tmpndo7bcvz
100%|██████████| 567/567 [00:00<00:00, 391339.54B/s]
INFO:tape.models.file_utils:copying /tmp/tmpndo7bcvz to cache at /home/ubuntu/.cache/torch/protein_models/6373bfcc1f9755cd1b90c75a4c82e5b6ace8db121253e86653b69c3fec08ed04.05edb4ed225e1907a3878f9d68b275d79e025b667555aa94a086e27cb5c591e0
INFO:tape.models.file_utils:creating metadata file for /home/ubuntu/.cache/torch/protein_models/6373bfcc1f9755cd1b90c75a4c82e5b6ace8db121253e86653b69c3fec08ed04.05edb4ed225e1907a3878f9d68b275d79e025b667555aa94a086e27cb5c591e0
INFO:tape.models.file_utils:removing temp file /tmp/tmpndo7bcvz
INFO:tape.models.modeling_utils:loading configuration file https://s3.amazonaws.com/proteindata/pytorch-models/bert-base-config.json from cache at /home/ubuntu/.cache/torch/protein_models/6373bfcc1f9755cd1b90c75a4c82e5b6ace8db121253e86653b69c3fec08ed04.05edb4ed225e19

tensor([[-1.7114e-01, -6.8594e-01, -8.5377e-01,  8.3713e-01,  4.9176e-01,
         -4.3934e-02, -6.3496e-01, -5.2483e-01, -2.3017e-01,  8.5670e-01,
          1.9850e+00, -9.6539e-01, -2.1384e-01,  1.3521e-01, -1.4059e-01,
          3.8717e-01,  7.6769e-01,  9.9204e-01,  9.9232e-01, -3.5369e-01,
          1.2552e+00, -8.8885e-01, -1.3965e+00, -9.3320e-01, -6.7184e-01,
         -3.5967e-01,  1.5412e+00, -8.2887e-01, -7.4518e-01,  9.3041e-01,
         -4.7263e-01, -1.4519e-01,  2.0215e+00, -1.7054e+00,  7.9398e-01,
          1.5622e+00,  3.7665e-01, -1.1410e+00, -9.9272e-01,  4.1447e-01,
          1.3132e-01, -9.2810e-01,  1.1416e+00, -1.2771e+00, -5.0179e-01,
         -1.0345e-01,  3.1901e-01,  3.8528e-01,  2.7052e-01, -1.9890e+00,
         -1.8634e+00,  6.9511e-01,  7.0682e-01,  2.4018e-01,  1.7369e-01,
         -6.2062e-01,  3.6796e-01,  2.5133e-01, -7.3765e-01,  8.1784e-02,
          5.0219e-01, -5.5550e-02, -5.1651e-01, -1.0116e+00, -5.7270e-01,
          5.4305e-01,  1.4781e-01, -7.

In [6]:
getUniRepPooledSequence(seq)

INFO:tape.models.file_utils:https://s3.amazonaws.com/proteindata/pytorch-models/unirep-base-config.json not found in cache, downloading to /tmp/tmpikm2nupg
100%|██████████| 285/285 [00:00<00:00, 155284.05B/s]
INFO:tape.models.file_utils:copying /tmp/tmpikm2nupg to cache at /home/ubuntu/.cache/torch/protein_models/f61813e6c3d93a4e696443982bbca4bfffa259c4abe29dfe680566d72861f512.903dc94e94ead9c24081473a281fe33b8f1f5df11db92cdc4b97bf4af5a50e10
INFO:tape.models.file_utils:creating metadata file for /home/ubuntu/.cache/torch/protein_models/f61813e6c3d93a4e696443982bbca4bfffa259c4abe29dfe680566d72861f512.903dc94e94ead9c24081473a281fe33b8f1f5df11db92cdc4b97bf4af5a50e10
INFO:tape.models.file_utils:removing temp file /tmp/tmpikm2nupg
INFO:tape.models.modeling_utils:loading configuration file https://s3.amazonaws.com/proteindata/pytorch-models/unirep-base-config.json from cache at /home/ubuntu/.cache/torch/protein_models/f61813e6c3d93a4e696443982bbca4bfffa259c4abe29dfe680566d72861f512.903dc94e94

tensor([[ 0.7857, -0.8269,  0.7735,  ...,  1.4782,  2.3078,  9.1452]],
       grad_fn=<CatBackward>)